<a href="https://colab.research.google.com/github/tanumeena28/Ml-Ass1Q1/blob/main/lab2q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [9]:
from google.colab import files
uploaded = files.upload()

Saving Housing.csv to Housing.csv


In [10]:
df = pd.read_csv('Housing.csv')

In [11]:
target_col = 'price'

In [12]:
def fill_with_zero(data):
    df_copy = data.copy()
    for col in df_copy.columns:
        if df_copy[col].dtype in [np.float64, np.int64]:
            df_copy[col] = [0 if pd.isna(x) else x for x in df_copy[col]]
    return df_copy


In [13]:
def fill_with_mean(data):
    df_copy = data.copy()
    for col in df_copy.columns:
        if df_copy[col].dtype in [np.float64, np.int64]:
            mean_val = df_copy[col].dropna().sum() / df_copy[col].dropna().shape[0]
            df_copy[col] = [mean_val if pd.isna(x) else x for x in df_copy[col]]
    return df_copy


In [14]:
def fill_with_median(data):
    df_copy = data.copy()
    for col in df_copy.columns:
        if df_copy[col].dtype in [np.float64, np.int64]:
            sorted_vals = sorted(df_copy[col].dropna())
            n = len(sorted_vals)
            median_val = (sorted_vals[n//2] if n % 2 != 0 else
                          (sorted_vals[n//2 - 1] + sorted_vals[n//2]) / 2)
            df_copy[col] = [median_val if pd.isna(x) else x for x in df_copy[col]]
    return df_copy

Normalization

In [15]:
def min_max_scale(data):
    df_copy = data.copy()
    for col in df_copy.columns:
        if df_copy[col].dtype in [np.float64, np.int64]:
            min_val = df_copy[col].min()
            max_val = df_copy[col].max()
            if min_val != max_val:
                df_copy[col] = [(x - min_val) / (max_val - min_val) for x in df_copy[col]]
            else:
                df_copy[col] = 0
    return df_copy

def standard_scale(data):
    df_copy = data.copy()
    for col in df_copy.columns:
        if df_copy[col].dtype in [np.float64, np.int64]:
            mean_val = df_copy[col].mean()
            std_val = df_copy[col].std()
            if std_val != 0:
                df_copy[col] = [(x - mean_val) / std_val for x in df_copy[col]]
            else:
                df_copy[col] = 0
    return df_copy


Outliers Remove

In [16]:
def remove_outliers_iqr(data):
    df_copy = data.copy()
    for col in df_copy.columns:
        if df_copy[col].dtype in [np.float64, np.int64]:
            Q1 = df_copy[col].quantile(0.25)
            Q3 = df_copy[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR
            df_copy = df_copy[(df_copy[col] >= lower) & (df_copy[col] <= upper)]
    return df_copy


Lenear regression

In [17]:
def linear_regression_manual(X, y):
    X = np.c_[np.ones(X.shape[0]), X]  # Add intercept
    theta = np.linalg.pinv(X.T @ X) @ X.T @ y
    y_pred = X @ theta
    mse = np.mean((y - y_pred) ** 2)
    return theta, mse

In [18]:
fill_strategies = {
    "zero": fill_with_zero,
    "mean": fill_with_mean,
    "median": fill_with_median
}

norm_strategies = {
    "minmax": min_max_scale,
    "standard": standard_scale
}

results = []

for fill_name, fill_func in fill_strategies.items():
    df_filled = fill_func(df.copy())
    for norm_name, norm_func in norm_strategies.items():
        df_norm = norm_func(df_filled.copy())

        for outlier_flag in [False, True]:
            df_exp = df_norm.copy()
            if outlier_flag:
                df_exp = remove_outliers_iqr(df_exp)

            # Drop rows missing target
            df_exp = df_exp.dropna(subset=[target_col])

            # Select numeric features only
            X = df_exp.drop(columns=[target_col]).select_dtypes(include=[np.number])
            y = df_exp[target_col].values

            if X.shape[0] > 0 and X.shape[1] > 0:
                theta, mse = linear_regression_manual(X.values, y)
                results.append({
                    "fill": fill_name,
                    "norm": norm_name,
                    "outliers_removed": outlier_flag,
                    "mse": mse
                })

In [19]:
results_df = pd.DataFrame(results).sort_values(by="mse")
print(results_df)

      fill      norm  outliers_removed       mse
1     zero    minmax              True  0.007382
5     mean    minmax              True  0.007382
9   median    minmax              True  0.007382
4     mean    minmax             False  0.011477
8   median    minmax             False  0.011477
0     zero    minmax             False  0.011477
7     mean  standard              True  0.281477
3     zero  standard              True  0.281477
11  median  standard              True  0.281477
2     zero  standard             False  0.437613
6     mean  standard             False  0.437613
10  median  standard             False  0.437613
